# Sports-Fashion Intersection Analysis

**Data-driven analysis of how sports and fashion intersect**

Focus areas:
1. The "Challengers" Effect on Tenniscore Fashion
2. Women's Sportswear Market Growth
3. World Cup 2026 Jersey Market Predictions

In [15]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Load Data

In [37]:
# Load processed datasets
data_path = Path('../data/processed')

tenniscore = pd.read_csv(data_path / 'tenniscore_processed.csv')
market = pd.read_csv(data_path / 'market_combined.csv')
trends = pd.read_csv(data_path / 'trends_processed.csv')
challengers = pd.read_csv(data_path / 'challengers_processed.csv')
summary = pd.read_csv(data_path / 'summary_stats.csv')

# Convert dates
trends['date'] = pd.to_datetime(trends['date'])
challengers['date'] = pd.to_datetime(challengers['date'])

print("Successfully loaded data")

Successfully loaded data


## 2. The Challengers Effect

### Analysis: How did Zendaya's film impact fashion trends?

In [38]:
# Top performing items
top_items = tenniscore.groupby('item')['yoy_growth_pct'].mean().sort_values(ascending=False).head(10)

print("Top 10 Tenniscore Items by YoY Growth:")
print("="*50)
for item, growth in top_items.items():
    print(f"{item:25} +{growth:.0f}%")

print(f"\nAverage growth across all items: +{tenniscore['yoy_growth_pct'].mean():.1f}%")

Top 10 Tenniscore Items by YoY Growth:
skorts                    +134%
cotton_jersey_jumpers     +126%
mini_pleated_skirts       +109%
white_trainers            +27%

Average growth across all items: +115.6%


In [39]:
# Demographic breakdown
demo_analysis = tenniscore.groupby('demographic')['demo_growth_pct'].agg(['mean', 'max', 'count']).sort_values('mean', ascending=False)

print("Growth by Demographic:")
print("="*50)
print(demo_analysis)

print("\nKey Insight: Gen Z leads with avg +{:.0f}% growth".format(demo_analysis.loc['gen_z', 'mean']))


Growth by Demographic:
                   mean    max  count
demographic                          
baby_boomer  126.333333  207.0      3
gen_z        110.000000  213.0      4
millennial   103.333333  126.0      3
gen_x         94.000000  142.0      3

Key Insight: Gen Z leads with avg +110% growth


In [40]:
# Challengers impact timeline
print("Challengers Impact Timeline:")
print("="*50)
print(challengers[['date', 'event', 'value']].to_string(index=False))

print("\nFilm released: April 26, 2024")
print(f"Total media value generated: ${summary['challengers_media_value_million'][0]:.1f}M")

Challengers Impact Timeline:
      date                         event  value
2024-03-01 Challengers press tour begins 1251.0
2024-04-01              Pre-release buzz  138.0
2024-04-26     Challengers movie release  245.0
2024-05-01    Peak social media activity 4500.0
2024-05-01                  Post-release   18.0
2024-05-01            Tennis dress sales   69.0
2024-05-01           Workout dress sales  114.0
2024-06-01            Sustained interest    6.0
2024-07-01               Wimbledon boost   42.5

Film released: April 26, 2024
Total media value generated: $42.5M


## 3. Market Growth Analysis

### Women's Sportswear Market Trajectory (2019-2033)

In [22]:
# Market growth summary
market_2023 = market[market['year'] == 2023]['market_value_billion_usd'].values[0]
market_2033 = market[market['year'] == 2033]['market_value_billion_usd'].values[0]
total_growth = ((market_2033 / market_2023) - 1) * 100

print(f"Women's Sportswear Market:")
print("="*50)
print(f"2023 Market Size: ${market_2023:.1f}B")
print(f"2033 Projection: ${market_2033:.0f}B")
print(f"Total Growth: +{total_growth:.0f}%")
print(f"CAGR: {summary['market_growth_rate'][0]:.1f}%")

Women's Sportswear Market:
2023 Market Size: $92.5B
2033 Projection: $250B
Total Growth: +170%
CAGR: 7.0%


In [23]:
# Jersey market analysis
jersey_2025 = market[market['year'] == 2025]['football_jersey_market_billion_usd'].values[0]
jersey_2026 = market[market['year'] == 2026]['football_jersey_market_billion_usd'].values[0]
wc_boost = ((jersey_2026 / jersey_2025) - 1) * 100

print(f"\nFootball Jersey Market:")
print("="*50)
print(f"2025 (Pre-WC): ${jersey_2025:.2f}B")
print(f"2026 (WC Year): ${jersey_2026:.2f}B")
print(f"World Cup Boost: +{wc_boost:.1f}%")


Football Jersey Market:
2025 (Pre-WC): $7.50B
2026 (WC Year): $8.20B
World Cup Boost: +9.3%


## 4. Google Trends Analysis

### Search Interest Over Time

In [41]:
# Peak search interest
print("Peak Search Interest:")
print("="*50)

for col in ['tenniscore', 'tennis_skirt', 'soccer_jersey', 'womens_activewear']:
    peak_idx = trends[col].idxmax()
    peak_date = trends.loc[peak_idx, 'date']
    peak_value = trends.loc[peak_idx, col]
    print(f"{col:20} Peak: {peak_value:3.0f} on {peak_date.strftime('%Y-%m-%d')}")

print(f"\nTenniscore peaked at {summary['tenniscore_search_peak'][0]:.0f} (post-Challengers)")

Peak Search Interest:
tenniscore           Peak:  79 on 2024-04-14
tennis_skirt         Peak:  63 on 2024-04-14
soccer_jersey        Peak:  62 on 2026-02-01
womens_activewear    Peak:  77 on 2026-01-18

Tenniscore peaked at 79 (post-Challengers)


In [25]:
# Pre vs Post Challengers
release_date = pd.Timestamp('2024-04-26')

pre_challengers = trends[trends['date'] < release_date]['tenniscore'].mean()
post_challengers = trends[
    (trends['date'] >= release_date) & 
    (trends['date'] < release_date + pd.Timedelta(days=90))
]['tenniscore'].mean()

challengers_lift = ((post_challengers / pre_challengers) - 1) * 100

print(f"\nChallengers Search Impact:")
print("="*50)
print(f"Pre-Release (avg): {pre_challengers:.1f}")
print(f"Post-Release (90d avg): {post_challengers:.1f}")
print(f"Lift: +{challengers_lift:.0f}%")


Challengers Search Impact:
Pre-Release (avg): 22.6
Post-Release (90d avg): 68.5
Lift: +204%


## 5. World Cup 2026 Predictions

### Soccer Jersey Trend Forecast

In [43]:
# Current trend (Feb 2026)
current_trend = trends[trends['date'] >= '2026-02-01']['soccer_jersey'].mean()

# Expected tournament spike
tournament_spike = 40  # % from research
expected_peak = current_trend * (1 + tournament_spike / 100)

print(f"World Cup 2026 Forecast:")
print("="*50)
print(f"Current Search Interest: {current_trend:.0f}")
print(f"Expected Tournament Spike: +{tournament_spike}%")
print(f"Predicted Peak Interest: {expected_peak:.0f}")
print(f"\nTournament dates: June 11 - July 19, 2026")
print(f"Host countries: USA, Canada, Mexico")

World Cup 2026 Forecast:
Current Search Interest: 62
Expected Tournament Spike: +40%
Predicted Peak Interest: 87

Tournament dates: June 11 - July 19, 2026
Host countries: USA, Canada, Mexico


## 6. Key Insights Summary

In [44]:
print("\n" + "="*60)
print("KEY INSIGHTS: SPORTS × FASHION INTERSECTION".center(60))
print("="*60)

print("\n1️⃣  THE CHALLENGERS EFFECT")
print("-" * 60)
print(f"   • Skorts led growth: +{tenniscore.groupby('item')['yoy_growth_pct'].mean().max():.0f}% YoY")
print(f"   • Gen Z adoption: +{tenniscore[tenniscore['demographic']=='gen_z']['demo_growth_pct'].max():.0f}%")
print(f"   • Media value: ${summary['challengers_media_value_million'][0]:.1f}M")
print(f"   • Cross-generational appeal across all age groups")

print("\n2️⃣  WOMEN'S SPORTSWEAR BOOM")
print("-" * 60)
print(f"   • Market size: ${summary['market_2024_billion'][0]:.1f}B → ${summary['market_2033_billion'][0]:.0f}B")
print(f"   • Growth rate: {summary['market_growth_rate'][0]:.1f}% CAGR")
print(f"   • Total growth: +{total_growth:.0f}% over 10 years")
print(f"   • Driven by athleisure and celebrity collabs")

print("\n3️⃣  WORLD CUP 2026 OPPORTUNITY")
print("-" * 60)
print(f"   • Current trend building: {current_trend:.0f} search interest")
print(f"   • Expected spike: +{tournament_spike}% during tournament")
print(f"   • Jersey market: ${jersey_2025:.1f}B → ${jersey_2026:.1f}B")
print(f"   • Retro 1994 WC nostalgia in designs")

print("\n" + "="*60)
print("CONCLUSION: Pop culture moments drive measurable,")
print("   sustained growth in sports fashion markets.")
print("="*60)


        KEY INSIGHTS: SPORTS × FASHION INTERSECTION         

1️⃣  THE CHALLENGERS EFFECT
------------------------------------------------------------
   • Skorts led growth: +134% YoY
   • Gen Z adoption: +213%
   • Media value: $42.5M
   • Cross-generational appeal across all age groups

2️⃣  WOMEN'S SPORTSWEAR BOOM
------------------------------------------------------------
   • Market size: $97.3B → $250B
   • Growth rate: 7.0% CAGR
   • Total growth: +170% over 10 years
   • Driven by athleisure and celebrity collabs

3️⃣  WORLD CUP 2026 OPPORTUNITY
------------------------------------------------------------
   • Current trend building: 62 search interest
   • Expected spike: +40% during tournament
   • Jersey market: $7.5B → $8.2B
   • Retro 1994 WC nostalgia in designs

CONCLUSION: Pop culture moments drive measurable,
   sustained growth in sports fashion markets.


## 7. Visuals for Article

In [45]:
# Chart 1: Tenniscore trend with Challengers marker
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=trends['date'],
    y=trends['tenniscore'],
    name='Tenniscore Search Interest',
    line=dict(color='#ff6b6b', width=3),
    fill='tozeroy'
))

# Add vertical line
fig1.add_shape(
    type="line",
    x0='2024-04-26',
    x1='2024-04-26',
    y0=0,
    y1=1,
    yref='paper',
    line=dict(color="orange", width=2, dash="dash")
)

# Add annotation
fig1.add_annotation(
    x='2024-04-26',
    y=1,
    yref='paper',
    text="Challengers Release",
    showarrow=False,
    yanchor='bottom',
    xanchor='right'
)

fig1.update_layout(
    title="The Challengers Effect: Tenniscore Search Explosion",
    xaxis_title="Date",
    yaxis_title="Search Interest (Indexed)",
    hovermode='x unified',
    height=400
)

fig1.show()

In [46]:
# Chart 2: Market growth projection
fig2 = go.Figure()

historical = market[market['segment'] != 'forecast']
forecast = market[market['segment'] == 'forecast']

fig2.add_trace(go.Scatter(
    x=historical['year'],
    y=historical['market_value_billion_usd'],
    name='Historical',
    mode='lines+markers',
    line=dict(color='#2ecc71', width=3)
))

fig2.add_trace(go.Scatter(
    x=forecast['year'],
    y=forecast['market_value_billion_usd'],
    name='Forecast',
    mode='lines+markers',
    line=dict(color='#3498db', width=3, dash='dash')
))

fig2.update_layout(
    title="Women's Sportswear: $92B → $250B by 2033",
    xaxis_title="Year",
    yaxis_title="Market Value (Billion USD)",
    height=400
)

fig2.show()

In [47]:
# Add vertical line
fig3.add_shape(
    type="line",
    x0='2026-06-11',
    x1='2026-06-11',
    y0=0,
    y1=1,
    yref='paper',
    line=dict(color="green", width=2, dash="dash")
)

# Add annotation
fig3.add_annotation(
    x='2026-06-11',
    y=1,
    yref='paper',
    text="World Cup Kickoff",
    showarrow=False,
    yanchor='bottom',
    xanchor='right'
)

## 8. Export for Article

Saving key stats and charts for my Substack article.

In [48]:
# Create summary for article
article_stats = {
    'challengers_impact': {
        'peak_growth': top_items.max(),
        'media_value_million': 42.5,
        'search_lift_pct': challengers_lift,
        'top_item': top_items.index[0]
    },
    'market_growth': {
        'market_2023_billion': market_2023,
        'market_2033_billion': market_2033,
        'cagr_pct': 7.0,
        'total_growth_pct': total_growth
    },
    'world_cup_2026': {
        'jersey_market_2026_billion': jersey_2026,
        'expected_spike_pct': tournament_spike,
        'current_search_interest': current_trend
    }
}

# Save as JSON for reference
import json
with open('../data/processed/article_stats.json', 'w') as f:
    json.dump(article_stats, f, indent=2)

print("successfully saved json")

successfully saved json
